In [3]:
f=open('../set-a/train/132539.txt')
dic={'time':-1,'Age':0,'Gender':1,'Height':2,'ICUType':3,'Weight':4,'Albumin':5,\
             'ALP':6,'ALT':7,'AST':8,'Bilirubin':9,'BUN':10,'Cholesterol':11,'Creatinine':12,\
             'DiasABP':13,'FiO2':14,'GCS':15,'Glucose':16,'HCO3':17,'HCT':18,'HR':19,\
             'K':20,'Lactate':21,'Mg':22,'MAP':23,'MechVent':24,'Na':25,'NIDiasABP':26,\
             'NIMAP':27,'NISysABP':28,'PaCO2':29,'PaO2':30,'pH':31,'Platelets':32,'RespRate':33,\
             'SaO2':34,'SysABP':35,'Temp':36,'TroponinI':37,'TroponinT':38,'Urine':39,'WBC':40}

mean=[0.0]*(len(dic)-1)
meancount=[0]*(len(dic)-1)
x=[]
times=[]
non_in_dic_count=0
count=0
age=gender=height=icutype=weight=-1
lastTime=0
totalData=[]
t_times=[]
for line in f.readlines():
    if count > 1:
        words=line.strip().split(",")
        timestamp=words[0]
        feature=words[1]
        value=words[2]
                    
        # 0 is missing value,orignl gender is 0/1 ,after preprocessing
        # gender is 0/1/2(missing,male,female)
        if timestamp == "00:00":
            if feature=='Age':
                age="0" if value=="-1" else value
                #calcuate mean
                if age !="0":
                    mean[dic[feature]]+=float(age)
                    meancount[dic[feature]]+=1
            if feature=='Gender':
                if value=="-1":
                    gender="0"
                if value=="0":
                    gender="1"
                if value=="1":
                    gender="2"
                #calcuate mean
                if gender !="0":
                    mean[dic[feature]]+=float(gender)
                    meancount[dic[feature]]+=1
            if feature=='Height':
                height="0" if value=="-1" else value
                #calcuate mean
                if height !="0":
                    mean[dic[feature]]+=float(height)
                    meancount[dic[feature]]+=1
            if feature == 'ICUType':
                icutype="0" if value=="-1" else value
                #calcuate mean
                if icutype !="0":
                    mean[dic[feature]]+=float(icutype)
                    meancount[dic[feature]]+=1
            if feature=='Weight':
                weight="0" if value=="-1" else value
                #calcuate mean
                if weight !="0":
                    mean[dic[feature]]+=float(weight)
                    meancount[dic[feature]]+=1
        else:
            if timestamp!=lastTime:
                data=[0.0]*(len(dic)-1)
                hourandminute=timestamp.split(":")
                t_times.append(float(hourandminute[0])*60+float(hourandminute[1]))
                data[0]=float(age)
                data[1]=float(gender)
                data[2]=float(height)
                data[3]=float(icutype)
                data[4]=float(weight)
                    
                data[dic[feature]]=float(value)
                mean[dic[feature]]+=float(value)
                meancount[dic[feature]]+=1
                    
                totalData.append(data)
            else:
                    
                totalData[-1][dic[feature]]=float(value)
                mean[dic[feature]]+=float(value)
                meancount[dic[feature]]+=1
                    
                    
        lastTime=timestamp      
    count+=1
                #if len(totalData)==24:
                #    break;
            
x.append(totalData)
times.append(t_times)
f.close()
def timeslicing(times, x, isSlicing,sliceGap):
        #slicing x, make time gap be 30min, get the average of 30min
        if not isSlicing:
            return
        else:
            newx=[]
            newtimes=[]
            for i in range(len(times)):
                nowx=x[i]
                nowtime=times[i]
                lasttime=0
                newnowx=[]
                newnowtime=[]
                count=[0.0]*(len(dic)-1)
                tempx=[0.0]*(len(dic)-1)
                #newnowx.append(tempx)
                #newnowtime.append(lasttime)
                nowtime.append(48*60+2)
                for j in range(len(nowtime)):
                    if nowtime[j]<=lasttime+sliceGap:
                        for k in range(0,len(dic)-1):
                            tempx[k]+=nowx[j][k]
                            if nowx[j][k]!=0:
                                count[k]+=1.0
                    else:
                        for k in range(0,len(dic)-1):
                            if count[k]==0:
                                count[k]=1.0
                            tempx[k]=tempx[k]/count[k]
                        while nowtime[j]>lasttime+sliceGap:
                            newnowx.append(tempx)
                            newnowtime.append(lasttime)
                            lasttime+=sliceGap
                            count=[0.0]*(len(dic)-1)
                            tempx=[0.0]*(len(dic)-1)
                        # j may be len(nowx), we add one point into nowtime before
                        if j<len(nowx):
                            for k in range(0,len(dic)-1):
                                tempx[k]+=nowx[j][k]
                                if nowx[j][k]!=0:
                                    count[k]+=1.0
                
                for j in range(len(newnowtime)):
                    if newnowx[j][0]==0:
                        newnowx[j][0]=nowx[0][0]
                    if newnowx[j][1]==0:
                        newnowx[j][1]=nowx[0][1]
                    if newnowx[j][2]==0:
                        newnowx[j][2]=nowx[0][2]
                    if newnowx[j][3]==0:
                        newnowx[j][3]=nowx[0][3]
                    if newnowx[j][4]==0:
                        newnowx[j][4]=nowx[0][4]
                            
                nowtime.pop(-1)
                newx.append(newnowx)
                newtimes.append(newnowtime)
            x=newx
            times=newtimes
        return x, times

In [4]:
x1, times1 = timeslicing(times, x, True, 60)

In [5]:
def normalization(x, std, mean, isNormal):
        if not isNormal:
            return
        for i in range(len(x)):
            for j in range(len(x[i])):
                for k in range(len(x[i][j])):
                    if x[i][j][k] != 0:
                        if std[k]==0:
                            x[i][j][k]=0.0
                        else:
                            x[i][j][k]=1.0/std[k]*(x[i][j][k]-mean[k])
        return x

In [6]:
import math
for i in range(len(mean)):
    if meancount[i]!=0:
        mean[i]=mean[i]/meancount[i]
# self.mean=mean
# normalization
m=[] # mask 0/1
# first calculate std
std=[0.0]*(len(dic)-1)
for onefile in x1:
    one_m=[]
    for oneclass in onefile:
        t_m=[0]*len(oneclass)
        for j in range(len(oneclass)):
            if oneclass[j] !=0:
                std[j]+=(oneclass[j]-mean[j])**2
                t_m[j]=1
        one_m.append(t_m)
    m.append(one_m)
for j in range(len(std)):
    if std[j] != 0 and meancount[j] != 1:
        std[j]=math.sqrt(1.0/(meancount[j]-1)*std[j])
        
isNormal=True
x = normalization(x1, std, mean, isNormal)



In [7]:
x_lengths=[] #
deltaPre=[] #time difference 
lastvalues=[] # if missing, last values
deltaSub=[]
subvalues=[]
for h in range(len(x)):
    # oneFile: steps*value_number
    oneFile=x[h]
    one_time=times[h]
    x_lengths.append(len(oneFile))
    
    one_deltaPre=[]
    one_lastvalues=[]
    
    one_deltaSub=[]
    one_subvalues=[]
    
    one_m=m[h]
    for i in range(len(oneFile)):
        t_deltaPre=[0.0]*len(oneFile[i])
        t_lastvalue=[0.0]*len(oneFile[i])
        one_deltaPre.append(t_deltaPre)
        one_lastvalues.append(t_lastvalue)
        
        if i==0:
            for j in range(len(oneFile[i])):
                one_lastvalues[i][j]=0.0 if one_m[i][j]==0 else oneFile[i][j]
            continue
        
        for j in range(len(oneFile[i])):
            if one_m[i-1][j]==1:
                one_deltaPre[i][j]=one_time[i]-one_time[i-1]
            if one_m[i-1][j]==0:
                one_deltaPre[i][j]=one_time[i]-one_time[i-1]+one_deltaPre[i-1][j]
                
            if one_m[i][j]==1:
                one_lastvalues[i][j]=oneFile[i][j]
            if one_m[i][j]==0:
                one_lastvalues[i][j]=one_lastvalues[i-1][j]

    for i in range(len(oneFile)):
        t_deltaSub=[0.0]*len(oneFile[i])
        t_subvalue=[0.0]*len(oneFile[i])
        one_deltaSub.append(t_deltaSub)
        one_subvalues.append(t_subvalue)
    #construct array 
    for i in range(len(oneFile)-1,-1,-1):    
        if i==len(oneFile)-1:
            for j in range(len(oneFile[i])):
                one_subvalues[i][j]=0.0 if one_m[i][j]==0 else oneFile[i][j]
            continue
        for j in range(len(oneFile[i])):
            if one_m[i+1][j]==1:
                one_deltaSub[i][j]=one_time[i+1]-one_time[i]
            if one_m[i+1][j]==0:
                one_deltaSub[i][j]=one_time[i+1]-one_time[i]+one_deltaSub[i+1][j]
                
            if one_m[i][j]==1:
                one_subvalues[i][j]=oneFile[i][j]
            if one_m[i][j]==0:
                one_subvalues[i][j]=one_subvalues[i+1][j]   
        
    
    #m.append(one_m)
    deltaPre.append(one_deltaPre)
    lastvalues.append(one_lastvalues)
    deltaSub.append(one_deltaSub)
    subvalues.append(one_subvalues)

In [221]:
deltaPre[0][2]

[60.0,
 60.0,
 90.0,
 60.0,
 90.0,
 90.0,
 90.0,
 90.0,
 90.0,
 90.0,
 90.0,
 90.0,
 90.0,
 90.0,
 90.0,
 90.0,
 90.0,
 90.0,
 90.0,
 60.0,
 90.0,
 90.0,
 90.0,
 90.0,
 90.0,
 90.0,
 60.0,
 60.0,
 60.0,
 90.0,
 90.0,
 90.0,
 90.0,
 60.0,
 90.0,
 90.0,
 90.0,
 90.0,
 90.0,
 60.0,
 90.0]

In [219]:
m[0][1]

[1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0]

In [220]:
m[0][2]

[1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0]

In [8]:
import numpy as np

x1 = np.arange(9.0).reshape(3,3)
x2 = np.arange(-1.0, 3.0).reshape(2, 2)
x2

array([[-1.,  0.],
       [ 1.,  2.]])

In [227]:
for i in range(10, -1, -1):
    print(i)

10
9
8
7
6
5
4
3
2
1
0


In [9]:
NB_FEATURES = 1
chlorine = np.loadtxt("../in/chlorine_m100.txt")
input = chlorine
mean = [0.0]*NB_FEATURES
meancount = [0]*NB_FEATURES
x = []
times = []

for row in input.transpose():
    # transpose input so each timeseries are rows

    t = 0
    totalData = []
    t_times = []
    for value in row:
        timestamp = t
        t_times.append(float(timestamp))
        totalData.append([float(value)])
        if not np.isnan(value):
            mean[0] += float(value)
            meancount[0] += 1
        t += (1)
        
    x.append(totalData)
    times.append(t_times)

for i in range(len(mean)):
    if meancount[i] != 0:
        mean[i] = mean[i]/meancount[i]
import math
isNormal = False
m = []

std = [0.0]*NB_FEATURES
for ts in x:
    ts_m = []
    for values in ts:
        v_m = ~np.isnan(values)
        if isNormal:
            for i in range(len(values)):
                if not np.isnan(value[i]):
                    std[i] += (value[i] - mean[i])**2
        ts_m.append(v_m)
    m.append(ts_m)

for i in range(len(std)):
    std[i] = math.sqrt(1.0/(meancount[i-1])*std[i])

# isNormal = isNormal


def normalization(x, mean, std,isNormal):
    new_x = []
    if not isNormal:
        return
    for ts in range(len(x)):
        new_ts = []
        for value in x[ts]:
            if value != np.nan:
                if std==0:
                    new_ts.append(0.0)
                else:
                    new_ts.append(1.0/std*(value-mean))
        new_x.append(new_ts)
    return new_x

normalization(x, mean, std, isNormal)

x_lengths = [] 
deltaPre = [] #time difference
lastvalues= [] # last observed value
deltaSub = [] # time difference in backward direction
subvalues = [] # last value in backward direction ?
for idx in range(len(x)):
    time_serie = x[idx]
    time = times[idx]
    x_lengths.append(len(time_serie))

    ts_deltaPre = []
    ts_lastvalues = []

    ts_deltaSub = []
    ts_subvalues = []

    ts_m = m[idx]

    for i in range(len(time_serie)):
        t_deltaSub = [0.0]*NB_FEATURES
        t_lastvalue = [0.0]*NB_FEATURES
        ts_deltaPre.append(t_deltaSub)
        ts_lastvalues.append(t_lastvalue)

        if i == 0:
            for j in range(len(time_serie[i])):
                ts_lastvalues[i][j]=0.0 if ts_m[i][j]==0 else time_serie[i][j]
            continue
        
        for j in range(len(time_serie[i])):
            if ts_m[i-1][j]==1: # if previous value was not a missing value
                ts_deltaPre[i][j]=time[i]-time[i-1]
            else: # else was missing
                ts_deltaPre[i][j] = time[i]-time[i-1] + ts_deltaPre[i-1][j]

            if ts_m[i][j]==1:
                ts_lastvalues[i][j] = time_serie[i][j]
            else:
                ts_lastvalues[i][j] = ts_lastvalues[i-1][j]
    

    for i in range(len(time_serie)):
        t_deltaSub = [0.0]*NB_FEATURES
        t_subvalues = [0.0]*NB_FEATURES
        ts_deltaSub.append(t_deltaSub)
        ts_subvalues.append(t_subvalues)
    # go in backward direction
    for i in range(len(time_serie)-1, -1, -1):
        if i == len(time_serie)-1:
            for j in range(len(time_serie[i])):
                ts_subvalues[i][j]=0.0 if ts_m[i][j]==0 else time_serie[i][j]
            continue
        for j in range(len(time_serie[i])):
            if ts_m[i+1][j] == 1:
                ts_deltaSub[i][j] = time[i+1] - time[i]
            else:
                ts_deltaSub[i][j] = time[i+1] - time[i] + ts_deltaSub[i+1][j]

        if ts_m[i][j] == 1:
            ts_subvalues[i][j]=time_serie[i][j]
        else:
            ts_subvalues[i][j] = ts_subvalues[i+1][j]

    deltaPre.append(ts_deltaPre)
    lastvalues.append(ts_lastvalues)
    deltaSub.append(ts_deltaSub)
    subvalues.append(ts_subvalues)

In [314]:
INPUT_LENGTH = 40

slice_start = 0
slice_end = INPUT_LENGTH
first_col = True
sliced_x = []
sliced_m = []
sliced_lengths = []
sliced_deltaPre = []
sliced_lastvalues = []
sliced_deltaSub = []
sliced_subvalues = []

x_arr = np.array(x)
m_arr = np.array(m)
x_lengths_arr = np.array(x_lengths)
deltaPre_arr = np.array(deltaPre)
lastvalues_arr =  np.array(lastvalues)
deltaSub_arr = np.array(deltaSub)
subvalues_arr = np.array(subvalues)

for col in range(x_arr.shape[0]):

    while slice_start < max(x_lengths):
        if first_col:
            x_slice = np.expand_dims(x_arr[col, slice_start:slice_end], axis=0)
            sliced_x = (x_slice)
            sliced_m = np.expand_dims(m_arr[col, slice_start:slice_end], axis=0)
            sliced_deltaPre = np.expand_dims(deltaPre_arr[col, slice_start:slice_end], axis=0)
            sliced_lastvalues = np.expand_dims(lastvalues_arr[col, slice_start:slice_end], axis=0)
            sliced_deltaSub = np.expand_dims(deltaSub_arr[col, slice_start:slice_end], axis=0)
            sliced_subvalues = np.expand_dims(subvalues_arr[col, slice_start:slice_end], axis=0)
            slice_start = slice_end
            slice_end += INPUT_LENGTH
            first_col = False

        x_slice = np.expand_dims(x_arr[col, slice_start:slice_end], axis=0)

        sliced_x = np.concatenate((sliced_x, x_slice), axis=0)
        sliced_m = np.concatenate((sliced_m, np.expand_dims(m_arr[col, slice_start:slice_end], axis=0)), axis=0)
        sliced_deltaPre = (np.concatenate((sliced_deltaPre, np.expand_dims(deltaPre_arr[col, slice_start:slice_end], axis=0)), axis=0))
        sliced_lastvalues = (np.concatenate((sliced_lastvalues, np.expand_dims(lastvalues_arr[col, slice_start:slice_end], axis=0)), axis=0))
        sliced_deltaSub = (np.concatenate((sliced_deltaSub, np.expand_dims(deltaSub_arr[col, slice_start:slice_end], axis=0)), axis=0))
        sliced_subvalues = (np.concatenate((sliced_subvalues, np.expand_dims(subvalues_arr[col, slice_start:slice_end], axis=0)), axis=0))

        slice_start = slice_end
        slice_end += INPUT_LENGTH
    slice_start = 0
    slice_end = INPUT_LENGTH
    # sliced_lengths = np.apply_along_axis(lambda x: len(x), 1, sliced_x).squeeze().tolist()


In [315]:
sliced_x.shape

(1250, 40, 1)

In [27]:
print((i-1)*32)
leftovers = len(sliced_x) - (i-1)*32
if leftovers > 0:
   print(i) 
    

1248
40


In [130]:
test = np.ones((32, 40, 1))

In [102]:
res = []
for i in range(40*32):
    res.append(test*i)
    i+=1
res= np.array(res)

In [118]:
res1 = res.T.reshape(-1, 50)

In [120]:
res.shape

(1280, 40)

In [148]:
test2 = np.concatenate((test, test*0), axis=0)
test2.shape

(32, 80, 1)

In [265]:
t = []
out = []
row = 0
col = 0
for ts in test:
    if col < 50:
        t.append(ts)
        col += 1
        continue
    if row == 0:
        out = np.array(t)
        t = []
        row += 1
        col = 0
        continue
    col = 0
    out = np.concatenate((out, t), axis =  0)
    t = []

    
        

In [ ]:
t

In [268]:
print(f'{row=}, {col=}')

row=0, col=32


In [221]:
for ts in test*0:
    if col < 50:
        t.append(ts)
        col += 1
        continue
    if row == 0:
        out = np.array(t)
        t=[]
        row += 1
        col = 0
        continue
    col = 0
    out = np.concatenate((out, t), axis = 1)
    t = []
for ts in test*2:
    if col < 50:
        t.append(ts)
        col += 1
        continue
    if row == 0:
        out = np.array(t)
        t=[]
        row += 1
        col = 0
        continue
    col = 0
    out = np.concatenate((out, t), axis = 1)
    t = []
for ts in test*3:
    if col < 50:
        t.append(ts)
        col += 1
        continue
    if row == 0:
        out = np.array(t)
        t =[]
        row += 1
        col = 0
        continue
    col = 0
    out = np.concatenate((out, np.array(t)), axis = 1)
    t = []

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 50 and the array at index 1 has size 100

In [273]:
for ts in test*3:
    if col < 50:
        t.append(ts)
        col += 1
        continue
    if row == 0:
        out = np.array(t)
        t = []
        row += 1
        col = 0
        continue
    col = 0
    out = np.concatenate((out, t), axis = 1)
    row += 1
    t = []

In [274]:
print(f'{row=}, {col=}')
out.shape

row=1, col=26


(50, 80, 1)

In [282]:
2 % 1

0

In [286]:
len = (1000/40)*50


In [292]:
for ts in x_arr:
    while slice_start < max(x_lengths):
        

27.77777777777778